In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 7.6MB/s 


In [ ]:
!pip install youtube_dl   

     |████████████████████████████████| 1.9MB 8.8MB/s 


In [1]:
import os
import sys
import pickle
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine

import cv2
import mtcnn

from tensorflow.keras.models import load_model
from utils import *

In [ ]:
path0 = "./data/"

# Готовая модель доступна по ссылке 
# https://drive.google.com/drive/folders/12aMYASGCKvDdkygSv1yQq8ns03AStDO_
# Необходимо скачать модель расположить в папке /data/
encoder_model = path0 + 'facenet_keras.h5'

train_dir = path0 + 'train'
encodings_path = path0 + 'encodings.pkl'
test_img_path = path0 + 'test/'
test_res_path = path0 + 'results/'

required_size = (160, 160)

In [ ]:
# Коэффициент точности распознавания лиц
# чем ниже коэффициент - тем более строгим будет отбор
recognition_t = 0.4

# Функции

In [ ]:
def get_rect(img, bbox=True):
    '''
    Функция получает на вход фотографию, детектирует на них лица, и определяет,
    принадлежат ли они Джиму.
    Затем возвращает изображение с обведенным лицом и булевую переменную:
    True - Джим
    False - кто-либо еще
    '''
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_detector.detect_faces(img_rgb)
    state = False
    for res in results:
        face, pt_1, pt_2 = get_face(img_rgb, res['box'])
        encode = get_encode(face_encoder, face, required_size)
        encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
        
        name = 'unknown'
        distance = float("inf")
        
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = 'Jim'
                distance = dist
        if name == 'unknown':
            if bbox:
                cv2.rectangle(img, pt_1, pt_2, (0,0, 255),1)
                cv2.putText(img,name, pt_1,cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1)
            else:
                pass
        else:
            if bbox:
                cv2.rectangle(img, pt_1, pt_2, (0, 255, 0),1)
                cv2.putText(img,name + f"__{distance:.2f}", pt_1 ,cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 1)
            state = True
    return img, state

In [2]:
def make_video(path, images):
    '''
    Функция записывает видео из полученных массивов
    '''
    video_writer = cv2.VideoWriter(path,
                                cv2.VideoWriter_fourcc(*'MPEG'), 
                                25.0, 
                                (frame_w, frame_h))

    # собираем все кадры в видео
    for i in range(len(images)):
        video_writer.write(images[i])

    # Ставим "точку", завершаем
    video_reader.release()
    video_writer.release()

# Нейронные сети

In [ ]:
# MTCNN - предобученный нейронный детектор лиц
face_detector = mtcnn.MTCNN()

In [ ]:
# В качестве энкодера используется предобученная нейросеть Keras-Facent 
# из репозитория nyoki-mtl/keras-facenet
face_encoder = load_model(encoder_model)

In [ ]:
encoding_dict = load_pickle(encodings_path)

# Проверка системы на видео

In [4]:
import youtube_dl
from IPython.display import YouTubeVideo
from moviepy.editor import *
from tqdm import tqdm_notebook

### Загружаем видео с ютуба, или добавляем в текущую дирректорию

In [1]:
# url = 'B0tzQZGDXOc'
# YouTubeVideo(url, start=460)

In [ ]:
!rm test.mp4 out.mp4

# Загружаем видео посвященное Джиму с youtube
!youtube-dl -f mp4 --output "Jim_full.%(ext)s" 'https://www.youtube.com/watch?v={url}'

# Для удобства вырежем одноминутный отрезок
!ffmpeg -i '/Jim_full.mp4' -s 360x250 -ss 00:01:00.00 -t 00:02:00.00 -c copy Jim.mp4

[youtube] B0tzQZGDXOc: Downloading webpage
[download] Destination: test.mp4
[download] 100% of 108.00MiB in 00:03
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --

### Запускаем систему на указанном видео

In [ ]:
# Путь к видео
video_ = '/Jim.mp4'

In [ ]:
# Создаем класс для чтения видео
video_reader = cv2.VideoCapture(video_)

nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
frame_h   = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_w   = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
fps       = int(video_reader.get(cv2.CAP_PROP_FPS))

images = []
wrong_imgs = []
screen_time = 0.

counter = 0 
while True:
    counter += 1
    if counter % 100 == 0:
        clear_output()
        print(round((counter / nb_frames) * 100, 2), '% processed')
    # Обрабатываем каждый фрейм видео
    stopper, img = video_reader.read()
    if not stopper:
        break
    # По каждому кадру делаем предикт
    image_pred, state = get_rect(img)
    # Если энкодер задетектировал лицо Джима - добавляем в соответствующий список
    if state:
        images.append(image_pred)
        screen_time += 1./fps
    if not state:
        wrong_imgs.append(image_pred)

### Кадры, на которых Facenet увидела Джима

In [ ]:
path = './predict.mp4'
make_video(path, images)

In [6]:
clip=VideoFileClip('./predict.mp4')
clip.ipython_display(width=640, center=False, fps=25, maxduration=100)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [2]:
print('Предположительное экранное время: %.2f сек.' % screen_time)

Предположительное экранное время: 3.68 сек.


### Кадры, отбракованные энкодером

In [ ]:
path = './wrong_images.mp4'
make_video(path, wrong_imgs)

In [2]:
clip=VideoFileClip('./wrong_images.mp4')
clip.ipython_display(width=640, center=False, fps=25, maxduration=100)